<a href="https://colab.research.google.com/github/rmontteiro/ML_IGTI_bootcamp/blob/master/Hands_on_Sintonia_Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hands-on Sintonia de Hiperparâmetros**

---

Sintonia de Hiperparâmetros com sklearn 

Vamos começar a olhar para alguns hiperparâmetros diversos e também para algumas tecnicas de otimização de hiperparâmetros.

Vamos seguir utilizando a base de dados **diabetes.csv** </a> que pode ser encontrada no OpenML em: https://www.openml.org/d/37. Essa base descreve o problema de predizer o resultado positivo ou negativo de um teste para diabetes aplicado em um conjunto específico de pacientes na Índia. 

Para a primeira etapa, vamos utilizar a divisão treino e teste e olhar para os dois dos três classificadores que usamos durante o hands-on de Classificação.
Note que, o processo é equivalente para as demais tarefas, tanto a de regressão quanto a de clusterização. Aqui, usaremos a classificação devido a simplicidade de interpretação do resultadoo. Mais informações sobre os classificadores podem ser encontradas na documentação do sklearn nos links abaixo. 

*   <a href = https://scikit-learn.org/stable/modules/svm.html> Support Vector Machine (SVM)</a>
*   <a href = https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html> Random Forest </a>


Na segunda etapa, vamos aplicar a técnica de cross validation para o processo de validação dos modelos e, em conjunto, vamos usar os métodos de grid search e random search para otimização dos hiperparâmetros. 


### *Importando* bibliotecas


In [13]:
import pandas as pd #biblioteca para manipulação de dados
import numpy as np #biblioteca para utilizacao de vetores e matrizes
import matplotlib.pyplot as plt #bibloteca para plotar graficos

In [14]:
#importando o nosso dataset
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes (1).csv


### Carregamento dos dados

In [15]:
#lendo o csv que contem a base de dados e armazanando em um df
df = pd.read_csv('diabetes.csv')

In [16]:
#imprimindo as 5 primeiras linhas do df para confirmação
df.head(5)

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,tested_positive
1,1,85,66,29,0,26.6,0.351,31,tested_negative
2,8,183,64,0,0,23.3,0.672,32,tested_positive
3,1,89,66,23,94,28.1,0.167,21,tested_negative
4,0,137,40,35,168,43.1,2.288,33,tested_positive


*   preg: number of times pregnant
*   plas: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
*   pres: Diastolic blood pressure (mm Hg)
*   skin: Triceps skin fold thickness (mm)
*   insu: 2-Hour serum insulin (mu U/ml)
*   mass: Body mass index (weight in kg/(height in m)^2)
*   pedi: Diabetes pedigree function
*   age: Age (years)
*   class: Class variable (tested_positive or tested_negative)

In [17]:
# Verificando o numero de amostras (linhas) e features (colunas) do dataset. 
print('Amostras e Features:', df.shape)

Amostras e Features: (768, 9)


In [18]:
# Verificando quais são os tipos das features
df.columns

Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age', 'class'], dtype='object')

### Pré processamento

Nesse problema, o único atributo categórico é a classe. Alguns modelos podem ter dificuldade de lidar com esse atributo. Nesse caso, precisamos transformar os valores de "tested_negative" e "tested_positivo" para 0 e 1.

Para isso, vamos fazer uma função bem simples. 

In [19]:
#criando um dicionario de dados para o mapeamento
name_to_class = {
    'tested_negative': 0,
    'tested_positive': 1
}

#substituindo os valores categóricos pelo mapeamento
df['class'] = df['class'].map(name_to_class)

#check
df.head(5)

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Uma outra etapa importante do pré-processamento consiste na avaliação de dados faltantes. 



In [20]:
# Analisando o resumo da base
df.describe()

,preg,plas,pres,skin,insu,mass,pedi,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Os modelos implementados no sklearn recebem como entrada para a modelagam um ou mais arrays. Dessa forma, precisamos modificar o df original para que seja possível a modelagem correta. 

Para isso, vamos separar o label das amotras, armazenar o nome das featues já que os arrays não fazem isso e depois retirar a coluna de labels do df original. Em seguida, vamos converter o df para array usando o numpy!

In [21]:
# armazenando os labels em um array
labels = np.array(df['class'])

# salvando a ordem das features
feature_list = list(df.columns)

In [22]:
# removendo a coluna de labels do df original
df = df.drop('class', axis = 1)

# check
df.columns

Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')

In [23]:
# convertendo df para array
data = np.array(df)

Agora estamos quase prontos para a modelagem em si!

Vamos usar a função <a href = http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html> train_test_split </a> e olhar para para algumas combinações de parâmetros dos modelos que selecionamos. 

In [24]:
# importar train_test_split do scikitlearn 
from sklearn.model_selection import train_test_split

# aplicando a funcao train_test_split para separar os conjuntos de treino e 
# teste segundo uma porcentagem de separação definida. 
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)

# Support Vector Machine SVM

Vamos dar uma olhada agora no comportamento do SVM

In [25]:
#importar o modelo SVM
from sklearn.svm import SVC

In [26]:
# treinando o SVM com kernel RBF
classifier1 = SVC(kernel='rbf').fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions1_labels = classifier1.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions1_labels})  
#p.head(10)

In [27]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions1_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions1_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions1_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions1_labels))

Matriz de Confusão
 [[106  17]
 [ 35  34]]

Acurácia
 0.7291666666666666

F1
 0.5666666666666667

AUCROC
 0.6772711205372923


Vamos tentar melhorar o SVM? 

Vamos trocar o kernel gaussiano pelo sigmoidal

In [28]:
# treinando o SVM com kernel sigmoidal
classifier2 = SVC(kernel='sigmoid').fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions2_labels = classifier2.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions2_labels})  
#p.head(10)

In [30]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions2_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions2_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions2_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions2_labels))

Matriz de Confusão
 [[94 29]
 [57 12]]

Acurácia
 0.5520833333333334

F1
 0.21818181818181817

AUCROC
 0.46907034287734173


Vamos tentar outra opção!


In [31]:
# treinando o SVM com kernel rbf, com largura da gaussiana alterada
classifier3 = SVC(kernel='rbf', gamma = 'auto').fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions3_labels = classifier3.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions3_labels})  
#p.head(10)

In [32]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions3_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions3_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions3_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions3_labels))

Matriz de Confusão
 [[123   0]
 [ 69   0]]

Acurácia
 0.640625

F1
 0.0

AUCROC
 0.5


Ultimo teste!

In [33]:
# treinando o SVM com kernel rbf, com largura da gaussiana alterada
classifier4 = SVC(C = 0.1, kernel='rbf', gamma = 0.0001).fit(train_data,train_labels)

# aplicando o modelo treinado 
predictions4_labels = classifier4.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions4_labels})  
#p.head(10)

In [34]:
#avaliando o modelo 
from sklearn import metrics
 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions4_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions4_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions4_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions4_labels))

Matriz de Confusão
 [[112  11]
 [ 43  26]]

Acurácia
 0.71875

F1
 0.49056603773584906

AUCROC
 0.6436903499469776


# Random Forest Classifiers


Vamos agora modelar o nosso problema utilizando o Random Forest Classifier e testar algumas combinações de hiperparâmetros!

In [35]:
# importar o modelo Random Forest Regressor
from sklearn.ensemble import RandomForestClassifier

# treinando o modelo 
classifier5 = RandomForestClassifier(n_estimators= 10, random_state=42).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions5_labels = classifier5.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions5_labels})  
#p.head(10)

In [36]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions5_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions5_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions5_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions5_labels))

Matriz de Confusão
 [[97 26]
 [24 45]]

Acurácia
 0.7395833333333334

F1
 0.6428571428571428

AUCROC
 0.72039589961117


Eita...

Será que o nosso resultado pode ser melhor?

Vamos trocar random_state e dar uma olhada em como a característica aleatoria do RF influencia no resultado.


In [37]:
# treinando o modelo 
classifier6 = RandomForestClassifier(n_estimators= 10, random_state = 124).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions6_labels = classifier6.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions5_labels})  
#p.head(10)

In [38]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions6_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions6_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions6_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions6_labels))

Matriz de Confusão
 [[100  23]
 [ 24  45]]

Acurácia
 0.7552083333333334

F1
 0.656934306569343

AUCROC
 0.7325910215623894


Vamos tentar aumentar o número de estimadores

In [39]:
# treinando o modelo 
classifier7 = RandomForestClassifier(n_estimators= 100, random_state = 42).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions7_labels = classifier7.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions5_labels})  
#p.head(10)

In [40]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions7_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions7_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions7_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions7_labels))

Matriz de Confusão
 [[96 27]
 [24 45]]

Acurácia
 0.734375

F1
 0.6382978723404256

AUCROC
 0.7163308589607635


In [41]:
# importar o modelo Random Forest Regressor
from sklearn.ensemble import RandomForestClassifier

# treinando o modelo 
classifier8 = RandomForestClassifier(n_estimators= 100, bootstrap = False, random_state = 42).fit(train_data, train_labels);

# aplicando o modelo treinado para a previsão do resultado do teste
predictions8_labels = classifier8.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
#p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions8_labels})  
#p.head(10)

In [42]:
#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels, predictions8_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels, predictions8_labels)) 
print('\nF1\n', metrics.f1_score(test_labels, predictions8_labels)) 
print('\nAUCROC\n', metrics.roc_auc_score(test_labels, predictions8_labels))

Matriz de Confusão
 [[93 30]
 [24 45]]

Acurácia
 0.71875

F1
 0.6250000000000001

AUCROC
 0.704135737009544


#### Qual seria o custo, isto é, qual o tempo que nós perdiaríamos ao ter que calibrar manualmente, de forma aleatória ou sistemática, todos os modelos que criamos? 

Só não!

Vamos dar uma olhada em como funciona o Random Search e o Grid Search aplicados? SIIIIIIIIIIIM

Vamos aproveitar para usar a validação cruzada estratificada!

O sklearn tem funções de Grid Search e Random Search implementadas que já fazem o processo de validação cruzada do modelo. 
Mais informações sobre esses métodos podem ser encontradas nos links abaixo!

*   <a href = https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV> Grid Search CV </a>
*   <a href = https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV> Random Search CV</a>

Vamos testa-las?

## Grid Search



**Vamos testar o Grid Search com kfold estratificado e o SVM**

In [43]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

In [44]:
#definindo hiperparâmetros
hiperparam = {'kernel':('sigmoid', 'rbf'), 'C':[0.01, 1, 10]}

#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#instânciando meu classificador
classifier = SVC()

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
grid_cv = GridSearchCV(classifier, hiperparam, cv = cv_strat, scoring = f1)
grid_cv.fit(data, labels)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 1, 10], 'kernel': ('sigmoid', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(f1_score), verbose=0)

In [45]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', grid_cv.cv_results_)


Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.01941767, 0.01399798, 0.01745899, 0.01294751, 0.0158699 ,
       0.01399415]), 'std_fit_time': array([0.00247034, 0.00083375, 0.00066229, 0.00128203, 0.00035979,
       0.00068607]), 'mean_score_time': array([0.00271089, 0.00211916, 0.00262506, 0.0020575 , 0.00251451,
       0.00201378]), 'std_score_time': array([5.49924792e-05, 1.08223243e-04, 1.46553442e-04, 2.06088137e-04,
       1.61005046e-04, 3.31328151e-04]), 'param_C': masked_array(data=[0.01, 0.01, 1, 1, 10, 10],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['sigmoid', 'rbf', 'sigmoid', 'rbf', 'sigmoid', 'rbf'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 0.01, 'kernel': 'sigmoid'}, {'C': 0.01, 'kernel': 'rbf'}, {'C': 1, 'kernel': 'sigmoid'}, {'C': 1, 'kernel': 'rbf'}, {'C'

In [46]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', grid_cv.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', grid_cv.best_params_)

print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', grid_cv.best_estimator_)

Melhor resultado f1: 0.6086487385278134


Melhor configuração de hiperparâmetros: {'C': 10, 'kernel': 'rbf'}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


**Vamos agora repetir o processo para o Random Forest**


In [47]:
#definindo hiperparâmetros
#hiperparam1 = {'n_estimators':[10, 100, 1000]}
hiperparam1 = {'n_estimators':[10, 100, 1000], 'bootstrap': (True, False)}
#hiperparam1 = {'n_estimators':[10, 100, 1000], 'bootstrap': (True, False), 'criterion': ('gini', 'entropy')}

#instânciando meu classificador
classifier1 = RandomForestClassifier()

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
grid_cv1 = GridSearchCV(classifier1, hiperparam1, cv = cv_strat, scoring = f1)
grid_cv1.fit(data, labels)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                        

In [48]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', grid_cv1.cv_results_)


Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.01893113, 0.17214801, 2.04730923, 0.01727967, 0.17152479,
       1.69476507]), 'std_fit_time': array([1.92132156e-03, 4.39901385e-03, 4.19814180e-01, 2.43490156e-04,
       5.58163019e-03, 2.18969505e-02]), 'mean_score_time': array([0.00214851, 0.00890315, 0.08860433, 0.00194104, 0.00905104,
       0.078528  ]), 'std_score_time': array([1.69846666e-04, 4.67356443e-04, 1.84105269e-02, 2.38192093e-05,
       3.94024696e-04, 2.09341191e-03]), 'param_bootstrap': masked_array(data=[True, True, True, False, False, False],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[10, 100, 1000, 10, 100, 1000],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'bootstrap': True, 'n_estimators': 10}, {'bootstrap': True, 'n_estimators': 100}, {'bootstrap': Tr

In [49]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', grid_cv1.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', grid_cv1.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', grid_cv1.best_estimator_)

Melhor resultado f1: 0.6341129418103595


Melhor configuração de hiperparâmetros: {'bootstrap': False, 'n_estimators': 1000}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


## Random Search

In [50]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


In [51]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions = dict(kernel = ['sigmoid', 'rbf'], 
                     C = uniform(loc=0, scale=10))

#instânciando meu classificador
classifier = SVC()

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_cv = RandomizedSearchCV(classifier, distributions, cv = cv_strat, scoring = f1, random_state = 42, n_iter = 10)
random_cv.fit(data, labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc2d9396a90>,
                                        'kernel': ['sigmoid', 'rbf']},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=make_scorer(f1_score),
                   verbose=0)

In [52]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', random_cv.cv_results_)

Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.01744556, 0.012888  , 0.01603594, 0.01621091, 0.01292675,
       0.01266408, 0.01329284, 0.01307423, 0.01327901, 0.01366773]), 'std_fit_time': array([0.00134174, 0.00112632, 0.00028591, 0.00021204, 0.00068382,
       0.00039942, 0.00020789, 0.00105814, 0.00024925, 0.00011624]), 'mean_score_time': array([0.00262587, 0.00192192, 0.00239663, 0.00238099, 0.00189824,
       0.00179586, 0.00175395, 0.00196502, 0.00178187, 0.00205679]), 'std_score_time': array([1.82977866e-04, 1.41435980e-04, 7.91623503e-05, 3.52593915e-05,
       4.09889732e-05, 2.23636627e-05, 3.54004947e-05, 2.42382745e-04,
       4.35082291e-05, 2.08751226e-05]), 'param_C': masked_array(data=[3.745401188473625, 1.834347898661638,
                   5.986584841970366, 4.458327528535912,
                   0.5808361216819946, 3.337086111390218,
                   7.080725777960454, 0.5641157902710026,
                   8.324426408004218, 0.007787658410

In [53]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', random_cv.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv.best_estimator_)

Melhor resultado f1: 0.6065838014605637


Melhor configuração de hiperparâmetros: {'C': 8.324426408004218, 'kernel': 'rbf'}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 SVC(C=8.324426408004218, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


Vamos testar agora o mesmo processo para o RF

In [54]:
from scipy.stats import randint

In [55]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions1 = dict(n_estimators = randint(10, 100),
                      bootstrap = [True, False],
                      criterion = ['gini', 'entropy'])

#instânciando meu classificador
classifier1 = RandomForestClassifier(random_state = 42)

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_cv1 = RandomizedSearchCV(classifier1, distributions1, cv = cv_strat, scoring = f1, random_state = 42, n_iter = 10)
random_cv1.fit(data, labels)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_s...
                     

In [56]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', random_cv1.cv_results_)

Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.05035028, 0.14366405, 0.15599177, 0.14227428, 0.05617263,
       0.12826819, 0.08965302, 0.16696312, 0.07163243, 0.070803  ]), 'std_fit_time': array([0.00172907, 0.0011314 , 0.00201038, 0.00106655, 0.00133373,
       0.00333197, 0.00289994, 0.00461311, 0.00180826, 0.00123763]), 'mean_score_time': array([0.00298197, 0.00645559, 0.00844219, 0.00736778, 0.0037457 ,
       0.00585859, 0.00426748, 0.00686109, 0.00440183, 0.00362468]), 'std_score_time': array([9.05384344e-05, 2.25649401e-04, 1.44383250e-03, 9.00251033e-05,
       8.54266245e-05, 2.43736983e-04, 1.33950231e-04, 2.51475713e-04,
       2.80180279e-05, 1.04730039e-04]), 'param_bootstrap': masked_array(data=[True, True, True, True, False, True, False, False,
                   False, False],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_crit

In [57]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', random_cv1.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv1.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv1.best_estimator_)

Melhor resultado f1: 0.6336649387264222


Melhor configuração de hiperparâmetros: {'bootstrap': False, 'criterion': 'gini', 'n_estimators': 42}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=42,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


O Processo de parametrização de um modelo nem sempre é simples. 

Existem alguns métodos que podem nos ajudar a suavizar o custo da otimização.

Entretanto, é preciso saber quais os hiperperâmetros mais interessantes para calibrar e qual a faixa de valores que faz mais sentido. 

Nesse hands-on falamos de Grid e Random Search. A escolha do método fica por sua conta. 
É preciso pensar com cuidado e avaliar as soluções encontradas pelos métodos. 

# Chegamos ao final do módulo! 

# Vejo vocês de novo em breve!